In [ ]:
#started 10.29.24
#this notebook was started post-sfn 2024
#the purpose is to "clean up" from previous notebooks pitx2_analysis_9.24 and pitx2_NWBDataExample
#the way of loading in and sorting data will be the same but analysis will be different 

In [1]:
from simply_nwb.pipeline import NWBSession
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
# MODIFIED FROM AB'S CODE - How all sessions are pulled from the filepath

#EXPERIMENTAL_ANIMALS = ["pitx005", "pitx013", "pitx014", "pitx015", "pitx016"]
#WT_ANIMALS = ["pitx010", "pitx012"]
SESSIONS_TO_PROCESS = list()
dirprefix = "C:\\Users\\minjarec\\OneDrive - The University of Colorado Denver\\Documents\\predict_nwbs"
for datename in os.listdir(dirprefix):
    datedir = os.path.join(dirprefix, datename)
    SESSIONS_TO_PROCESS.append(datedir)

crystalSessions = list()
for session in SESSIONS_TO_PROCESS:
    sessionToAppend = NWBSession(session)
    crystalSessions.append(sessionToAppend)

In [3]:
# MODIFIED FROM AB'S CODE - creates lists of sessions according to dosage

cnoSessions05 = list()
cnoSessions2 = list()
salineSessions = list()
for session in crystalSessions:
    sessionID = session.nwb.session_description #this is getting the dosage from the .txt file containing mouse info
    if sessionID == "Dosage 0.5mg/kg":
        cnoSessions05.append(session)
    elif sessionID == "Dosage 2mg/kg":
        cnoSessions2.append(session)
    elif sessionID == "Dosage 0mg/kg":
        salineSessions.append(session)
    else:
        print("fuck") #lol thanks Anna <3
print('There are', len(cnoSessions05), ' 0.5mg/kg sessions')
print('There are', len(cnoSessions2), '2.0mg/kg sessions')
print('There are', len(salineSessions), '0mg/kg sessions')
print('There are', len(crystalSessions), 'total sessions')

There are 40  0.5mg/kg sessions
There are 46 2.0mg/kg sessions
There are 41 0mg/kg sessions
There are 127 total sessions


In [ ]:
# MODIFIED FROM AB'S CODE - looking at raw eye traces 

#%pyplot qt # this is for 
#sess = cnoSessions05[4]
sess = cnoSessions2[9]
#sess = salineSessions[20]
#sess2 = sessionsNR1KO1[2]
filtered = sess.pull("PutativeSaccades.pose_reoriented")
#filtered2 = sess2.pull("PutativeSaccades.pose_reoriented")
#x = (filtered[2000:5000][:, 0]) #cnosessions05
x = (filtered[000:8000][:, 0]) #cnosessions2
#x = (filtered[0000:8000][:, 0]) #salinesessions05
#x2 = (filtered2[0:40000][:, 0])
#plt.plot(x, color='slateblue') #cnosessions05
plt.plot(x, color='darkslateblue') #cnosessions2
#plt.plot(x, color='mediumslateblue') #salinesessions
#plt.plot(x2, color='r')

In [ ]:
#from Spencer - sorting animals 

animal_sessions = {}

# Sort and group sessions by animal_id and dosage
for session in crystalSessions:
    animal_id = session.nwb.subject.subject_id  # Get the animal_id
    sessionID = session.nwb.session_description  # Get the dosage

    #Print the extracted animal_id and dosage
    #print(f"Session: Animal ID: {animal_id}, Dosage: {sessionID}")

    # Map sessionID to a dosage key
    dosage = None
    if sessionID == "Dosage 0.5mg/kg":
        dosage = "0.5mg/kg"
    elif sessionID == "Dosage 2mg/kg":
        dosage = "2mg/kg"
    elif sessionID == "Dosage 0mg/kg":
        dosage = "0mg/kg"
# ( aid in animals) AND (dosage == true)  # a = "" -> false a = "asdjasflajs" -> true
    # Add the session to the corresponding animal_id and dosage group
    if animal_id not in animal_sessions:
        animal_sessions[animal_id] = {}
    if dosage not in animal_sessions[animal_id]:
        animal_sessions[animal_id][dosage] = []
    
    animal_sessions[animal_id][dosage].append(session)

# Debugging: Check the count of sessions grouped by animal and dosage
for animal_id, dosages in animal_sessions.items():
    for dosage, sessions in dosages.items():
        print(f"Animal {animal_id}, Dosage '{dosage}': {len(sessions)} sessions")

dosages = ["0mg/kg", "0.5mg/kg", "2mg/kg"]
directions = ["nasal", "temporal"]
print(animal_sessions)
# { "nasal": {
#              "0mg/kg":{ 
#                         "velocities": [..],  # (saccadenum,) a single number list [1.4, ..]
#                         "amps": [..], 
#                         "durations: [...},  # (saccadenum, start/stop) [[start, stop], [start, stop], ..] 
#                         "times: [...]
#               }, 
#               "0.5mg/kg": {{...}, ...},
#               <other dosages, etc>
#           }
# "temporal: {
#           <dosages>:<vels, amps, etc>, ...
#           }
# }

# amp_vels_durations_by_direction_and_dosage["nasal"]["0mg/kg"]["velocities"]
# amp_vels_durations_by_direction_and_dosage["nasal"]["0mg/kg"]["animal_name"]
print("="*50)
def calc_our_data():
    amp_vels_durations_by_direction_and_dosage = {}
    for animal_id in animal_sessions.keys():
        for dosage in dosages:
            print(f"Available dosages for animal '{animal_id}': '{list(animal_sessions[animal_id].keys())}'")
            animal_dosage_sessions = animal_sessions[animal_id][dosage]
            for sess in animal_dosage_sessions:
                for direc in directions:
                    all_waveforms = sess.pull(f"PredictSaccades.saccades_predicted_{direc}_waveforms")
                    epochs = sess.pull(f"PredictSaccades.saccades_predicted_{direc}_epochs")
                    peaks = sess.pull(f"PredictSaccades.saccades_predicted_{direc}_peak_indices")
                    # print(f"{animal_id}-{dosage}-{direc} wavs shape {waveforms.shape}")
                    # print(f"{animal_id}-{dosage}-{direc} epochs shape {epochs.shape}")
                    
                    amps = []
                    durations = []
                    vels = []
                    saccade_times = []
                    
                    for saccadenum in range(all_waveforms.shape[0]):
                        start_and_stop = (epochs[saccadenum] - peaks[:, None][saccadenum]) + (all_waveforms.shape[1]/2)  # time axis shape = 80 divide by 2 is 40 (since saccade is centered)
                        start = int(start_and_stop[0])
                        stop = int(start_and_stop[1])
                        waveforms = all_waveforms[saccadenum, start:stop]
                        
                        amp = np.max(waveforms) - np.min(waveforms)
                        amps.append(amp)
                        duration = (epochs[saccadenum][1] - epochs[saccadenum][0])/200  # Duration in frames
                        vels.append(amp / duration)
                        durations.append(duration)
                        saccade_times.append(epochs[saccadenum][0])
                    if direc not in amp_vels_durations_by_direction_and_dosage:
                        amp_vels_durations_by_direction_and_dosage[direc] = {}
                    if dosage not in amp_vels_durations_by_direction_and_dosage[direc]:
                        amp_vels_durations_by_direction_and_dosage[direc][dosage] = {}
                    
                    animal_names_list = [animal_id]*all_waveforms.shape[0]  # Create a repeated animal name list like ["samename", "samename", repeats with same data..]
                    # The repeated names are a bit redundant but useful to match a specific saccade to an animal name
                    for varname, vardata in [("velocity", vels), ("amplitude", amps), ("saccade_times", saccade_times), ("durations", durations), ("animal_name", animal_names_list)]:
                        # Extend takes two lists like [1,2,3] and [4,5,6] into [1,2,3,4,5,6]
                        # APPENDING DOES [1,2,3].append([4,5,6]) -> [1,2,3, [4,5,6] ]
                        if varname not in amp_vels_durations_by_direction_and_dosage[direc][dosage]:
                            amp_vels_durations_by_direction_and_dosage[direc][dosage][varname] = []
                        amp_vels_durations_by_direction_and_dosage[direc][dosage][varname].extend(vardata)
    
    return amp_vels_durations_by_direction_and_dosage

amp_vels_durations_by_direction_and_dosage = calc_our_data()

print("="*50)
for direc in ["nasal", "temporal"]:
    for dosage in ["0mg/kg", "0.5mg/kg", "2mg/kg"]:
        for varname in ["velocity", "amplitude", "saccade_times", "durations", "animal_name"]:
            amp_vels_durations_by_direction_and_dosage[direc][dosage][varname] = np.array(amp_vels_durations_by_direction_and_dosage[direc][dosage][varname])
            print(f"{direc}-{dosage}-{varname} {amp_vels_durations_by_direction_and_dosage[direc][dosage][varname].shape}")
print("="*50)


